<center><h1>Nvidia's End-to-End Deep Learning for Self-Driving Cars</h1></center>

In [51]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from cv2 import cv2
from pathlib import Path

from torch.utils.data import Dataset, DataLoader
import torch

from albumentations.pytorch.transforms import ToTensorV2

In [16]:
root = Path("./driving_dataset/")

In [17]:
all_paths_gen = root.glob("*")

In [18]:
all_paths = list(all_paths_gen)

In [19]:
label_path = "./driving_dataset/data.txt"

## Steering Dataset

In [61]:
class SteerDataset(Dataset):
    """
    Dataset to read images and corresponding steering angles.
    """
    def __init__(
        self, 
        label_path='./driving_dataset/data.txt',
        transforms=None
    ):
        self.paths, self.angles = self.get_images_angles(label_path)
        self.transforms = transforms
        
    def get_images_angles(self, label_path):
        paths, angles = [], []
        with open(label_path, "r") as f:
            for line in f.readlines():
                image, angle = line.split(" ")
                full_path = f"./driving_dataset/{image}"
                paths.append(full_path)
                angles.append(float(angle))
                
        return paths, angles
                
    def __getitem__(self, idx):
        image = cv2.imread(self.paths[idx])
        angle = self.angles[idx]
        
        # Apply transforms
        if self.transforms is not None:
            transformed_image = self.transforms(image=image)
            image = transformed_image['image']
            
        # Convert image to torch.tensor
        image_transform = ToTensorV2()(image=image)
        image = image_transform['image']
        
        # Convert angle into a torch.tensor
        angle = torch.tensor(angle, dtype=torch.float32)
        
        return {
            "image": image,
            "angle": angle
        }
    
    def __len__(self):
        return len(self.paths)

In [56]:
ds = SteerDataset()

In [59]:
ds[100]

{'image': tensor([[[168, 171, 170,  ..., 205, 205, 206],
          [169, 170, 165,  ..., 202, 182, 209],
          [169, 171, 165,  ..., 197, 212, 207],
          ...,
          [ 63,  59,  68,  ...,  61,  72,  67],
          [ 54,  76,  66,  ...,  57,  63,  73],
          [ 63,  65,  69,  ...,  62,  60,  59]],
 
         [[167, 173, 177,  ..., 220, 214, 202],
          [170, 174, 174,  ..., 221, 194, 212],
          [173, 178, 176,  ..., 223, 233, 219],
          ...,
          [ 67,  61,  68,  ...,  56,  68,  63],
          [ 55,  76,  58,  ...,  63,  68,  72],
          [ 63,  61,  61,  ...,  73,  73,  59]],
 
         [[193, 197, 196,  ..., 223, 218, 207],
          [191, 193, 188,  ..., 224, 196, 216],
          [184, 187, 184,  ..., 223, 231, 219],
          ...,
          [ 86,  79,  82,  ...,  58,  73,  68],
          [ 69,  88,  68,  ...,  62,  69,  74],
          [ 75,  72,  68,  ...,  71,  71,  59]]], dtype=torch.uint8),
 'angle': tensor(2.1200)}